## 1. Importar Bibliotecas

In [38]:
import pandas as pd
import json
from collections import Counter
import requests
import csv
import os

In [46]:
# Criar diretórios necessários
os.makedirs("../data/input", exist_ok=True)
os.makedirs("../data/output", exist_ok=True)
print("Diretórios criados/verificados: data/input, data/output")

Diretórios criados/verificados: data/input, data/output


## 2. Coleta de Dados da API

In [40]:
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjce/_search"
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="

payload = json.dumps({
    "size": 10000,
    "query": {"match": {"assuntos.codigo": "12487"}},
    "sort": [{"dataAjuizamento": {"order": "desc"}}]
})

headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
dados_dict = response.json()
print(f"Total de processos encontrados na API: {dados_dict['hits']['total']['value']}")
print(f"Processos retornados nesta consulta: {len(dados_dict['hits']['hits'])}")

# Salvando em csv (números de processos para referência)
processos = dados_dict["hits"]["hits"]

numeros = []
for processo in processos:
    numero_processo = processo["_source"]["numeroProcesso"]
    numeros.append(numero_processo)

Total de processos encontrados na API: 6046
Processos retornados nesta consulta: 6046


## 3. Processamento dos Dados (Python Dataframe)

In [47]:
# Extrair dados dos processos
processos = []

for hit in dados_dict['hits']['hits']:
    processo = hit['_source']
    numero_processo = processo['numeroProcesso']
    grau = processo['grau']
    classe = processo['classe']['nome']
    assuntos = processo['assuntos']
    data_ajuizamento = processo['dataAjuizamento']
    ultima_atualizacao = processo['dataHoraUltimaAtualizacao']
    formato = processo['formato']['nome']
    codigo = processo['orgaoJulgador']['codigo']
    orgao_julgador = processo['orgaoJulgador']['nome']
    municipio = processo['orgaoJulgador']['codigoMunicipioIBGE']
    
    try:
        movimentos = processo['movimentos']
    except:
        movimentos = []
    
    processos.append([numero_processo, classe, data_ajuizamento, ultima_atualizacao, formato,
                     codigo, orgao_julgador, municipio, grau, assuntos, movimentos])

df = pd.DataFrame(processos, columns=['numero_processo', 'classe', 'data_ajuizamento', 'ultima_atualizacao',
                                      'formato', 'codigo', 'orgao_julgador', 'municipio', 'grau', 'assuntos', 'movimentos'])

print(f"Total de processos no DataFrame: {len(df)}")
df.sample(5)

Total de processos no DataFrame: 6046


,numero_processo,classe,data_ajuizamento,ultima_atualizacao,formato,codigo,orgao_julgador,municipio,grau,assuntos,movimentos
391,02243603320248060001,Apelação Cível,20250221000000,2025-09-10T20:38:16.810Z,Eletrônico,88592,GADES - MARCOS WILLIAM LEITE DE OLIVEIRA,2304400,G2,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 3, 'val..."
3504,02896523320228060001,Cumprimento Provisório de Sentença,20221123000000,2024-08-20T13:00:06.093Z,Eletrônico,8701,3ª VARA DA INFANCIA E JUVENTUDE DA COMARCA DE ...,2304400,G1,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'codigo': 1061, 'nome': 'Disponibilização no..."
3827,06365997520228060000,Agravo de Instrumento,20220928000000,2025-08-16T11:03:54.895Z,Eletrônico,50880,GADES - EMANUEL LEITE ALBUQUERQUE,2304400,G2,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 19, 'va..."
2546,02460696120238060001,Cumprimento de sentença,20230711000000,2024-10-09T10:14:45.247Z,Eletrônico,8701,3ª VARA DA INFANCIA E JUVENTUDE DA COMARCA DE ...,2304400,G1,"[{'codigo': 12937, 'nome': 'Direitos da Person...","[{'complementosTabelados': [{'codigo': 4, 'val..."
2985,02191304420238060001,Procedimento Comum Cível,20230329000000,2024-08-20T13:23:23.876Z,Eletrônico,8479,17ª VARA CIVEL DA COMARCA DE FORTALEZA,2304400,G1,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 3, 'val..."


## 4. Análise de Decisões
#### Aqui podemos restringir os termos para "procedente/improcedente" ou expandir

In [48]:
# Extrair decisões dos movimentos
decisoes_por_processo = []
tipos_decisao_contagem = []

for idx, row in df.iterrows():
    numero = row['numero_processo']
    movimentos = row['movimentos']
    grau = row['grau']
    
    decisoes_encontradas = []

    if movimentos:
        for mov in movimentos:
            nome_mov = mov.get('nome', '')
            
            termos_decisao = [
                "Procedência",
                "Improcedência",
                "Improcedência do pedido e improcedência do pedido contraposto"
            ]
            
            # Identificar decisões
            if (any(palavra in nome_mov for palavra in termos_decisao) and grau == 'G1') :
                decisoes_encontradas.append(nome_mov)
                tipos_decisao_contagem.append(nome_mov)

    if decisoes_encontradas:
        decisoes_por_processo.append({
            'numero_processo': numero,
            'decisoes': decisoes_encontradas
        })

print(f"Processos com decisões: {len(decisoes_por_processo)} de {len(df)}")
print(f"\nTipos de decisões encontradas:")
for tipo, count in Counter(tipos_decisao_contagem).most_common(10):
    print(f"  {tipo}: {count}")

Processos com decisões: 1799 de 6046

Tipos de decisões encontradas:
  Procedência: 1284
  Procedência em Parte: 259
  Improcedência: 222
  Procedência em parte do pedido e improcedência do pedido contraposto: 30
  Improcedência do pedido e improcedência do pedido contraposto: 14
  Procedência do pedido e improcedência do pedido contraposto: 5
  Procedência do Pedido - Reconhecimento pelo réu: 3
  Procedência em parte do pedido e procedência do pedido contraposto: 1
  Procedência do pedido e procedência do pedido contraposto: 1


In [49]:
# Criar DataFrame de decisões
decisoes_lista = []

for item in decisoes_por_processo:
    for decisao in item['decisoes']:
        decisoes_lista.append({
            'numero_processo': item['numero_processo'],
            'tipo_decisao': decisao,
        })

df_decisoes = pd.DataFrame(decisoes_lista)
df_decisoes = df_decisoes.drop_duplicates(subset='numero_processo', keep='first')
df_decisoes.head(10)

,numero_processo,tipo_decisao
0,30039390220258060071,Improcedência
1,02187331420258060001,Procedência
2,30465508320258060001,Procedência em Parte
3,30415587920258060001,Procedência em Parte
4,30048329320258060167,Procedência
5,02162085920258060001,Procedência
6,30355412720258060001,Procedência
7,30022788520258060071,Procedência
8,30303751420258060001,Procedência em Parte
9,30292622520258060001,Procedência


## 5. Seleção Estratificada de Registros
#### Pegar todos os "Improcedência" (259 + 14) e 273 "Procedência" de forma distribuída

In [50]:
# Separar decisões por tipo
df_procedencia = df_decisoes[df_decisoes['tipo_decisao'] == 'Procedência'].copy()
df_improcedencia = df_decisoes[df_decisoes['tipo_decisao'] == 'Improcedência'].copy()
df_improcedencia_contraposto = df_decisoes[df_decisoes['tipo_decisao'] == 'Improcedência do pedido e improcedência do pedido contraposto'].copy()

# Registros de Improcedência (236 registros: 222 + 14)
df_improcedencias_todos = pd.concat([df_improcedencia, df_improcedencia_contraposto])

# Seleção estratificada de Procedência (361 registros)
procedencias = []
tamanho_bloco = 20
pulo = 50
posicao = 0
total_procedencia = len(df_procedencia)

while len(procedencias) <= 361 and posicao < total_procedencia:
    # Pegar um bloco de 20 registros
    fim_bloco = posicao + tamanho_bloco
    # Colocar índices de i variando de posicao até fim_bloco na lista "procedencias"
    procedencias.extend(range(posicao, fim_bloco))

    # Pular 100 registros
    posicao = fim_bloco + pulo
    
# Garantir que temos exatamente 361 ao invés de 380
procedencias = procedencias[:361]

# Selecionar os registros de Procedência
df_procedencia_selecionada = df_procedencia.iloc[procedencias].copy()

# Combinar todos os registros selecionados
df_decisoes_balanceado = pd.concat([df_procedencia_selecionada, df_improcedencias_todos])

print(f"\nTotal de registros no dataset balanceado: {len(df_decisoes_balanceado)}")
print(f"  - Procedência: {len(df_procedencia_selecionada)}")
print(f"  - Improcedência: {len(df_improcedencia)}")
print(f"  - Improcedência do pedido e improcedência do pedido contraposto: {len(df_improcedencia_contraposto)}")

# Mostrar distribuição
df_decisoes_balanceado.head(10)


Total de registros no dataset balanceado: 593
  - Procedência: 361
  - Improcedência: 219
  - Improcedência do pedido e improcedência do pedido contraposto: 13


,numero_processo,tipo_decisao
1,02187331420258060001,Procedência
4,30048329320258060167,Procedência
5,02162085920258060001,Procedência
6,30355412720258060001,Procedência
7,30022788520258060071,Procedência
9,30292622520258060001,Procedência
13,30012724320258060071,Procedência
14,30193154420258060001,Procedência
15,30186884020258060001,Procedência
16,30182752720258060001,Procedência


## 6. Exportar Resultados

In [51]:
# Salvar DataFrames em CSV
cwd = os.getcwd()

df.to_csv('../data/output/processos_completo.csv', sep=',', header=True, index=False)
df_decisoes_balanceado.to_csv('../data/output/decisoes_resumo.csv', sep=',', header=True, index=False)

# Salvar números de processos balanceados em CSV
numeros_balanceados = df_decisoes_balanceado['numero_processo'].unique()
with open('../data/output/numeros_processos.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['numeroProcesso'])
    for num in numeros_balanceados:
        writer.writerow([num])

# Salvar dados completos em JSON (apenas processos balanceados)
processos_balanceados = [p for p in dados_dict['hits']['hits'] if p['_source']['numeroProcesso'] in numeros_balanceados]
dados_completos_balanceados = {
    'hits': {
        'hits': processos_balanceados,
        'total': {'value': len(processos_balanceados)}
    }
}

with open('../data/output/dados_completos.json', 'w', encoding='utf-8') as f:
    json.dump(dados_completos_balanceados, f, indent=2, ensure_ascii=False)

# Salvar processos com decisões
with open('../data/output/processos_com_decisoes.json', 'w', encoding='utf-8') as f:
    json.dump(decisoes_por_processo, f, indent=2, ensure_ascii=False)

print("Arquivos exportados:")
print(f"  - data/output/processos_completo.csv")
print(f"  - data/output/decisoes_resumo.csv ({len(df_decisoes_balanceado)} registros balanceados)")
print(f"  - data/output/numeros_processos.csv ({len(numeros_balanceados)} processos únicos)")
print(f"  - data/output/dados_completos.json ({len(processos_balanceados)} processos balanceados)")
print(f"  - data/output/processos_com_decisoes.json")

Arquivos exportados:
  - data/output/processos_completo.csv
  - data/output/decisoes_resumo.csv (593 registros balanceados)
  - data/output/numeros_processos.csv (593 processos únicos)
  - data/output/dados_completos.json (754 processos balanceados)
  - data/output/processos_com_decisoes.json


## 7. Próximos Passos: Coleta Completa de Dados

Após gerar os arquivos básicos acima, execute o pipeline completo de coleta de dados:

### Opção 1: Pipeline Completo (3 etapas)
```bash
python coletar_dados_ml.py
```

Isso executará:
1. **Scraping** - Coleta juiz e requerente do TJCE
2. **Inferência de Sexo** - Infere sexo dos nomes
3. **Geração de Features** - Gera dataset ML com 29 features

### Opção 2: Executar Etapas Individuais
```bash
# Apenas scraping
python coletar_dados_ml.py --etapa scraping

# Apenas inferência de sexo
python coletar_dados_ml.py --etapa inferir_sexo

# Apenas geração de features
python coletar_dados_ml.py --etapa features
```

### Arquivos Gerados
- `data/dados_processos_tjce.csv` - Dados do scraping
- `data/dados_processos_com_sexo.csv` - Dados + sexo inferido
- `data/dataset_ml_completo.csv` - Dataset final (29 features)